In [59]:
#!wget https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx
#!wget https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx.data


## Question 1: Knowing the output node

In [60]:
!pip install onnxruntime

import onnxruntime as ort

onnxmodelpath = 'hair_classifier_v1.onnx'
session = ort.InferenceSession(onnxmodelpath)

inputs = session.get_inputs()
outputs = session.get_outputs()

input_name = inputs[0].name
output_name = outputs[0].name


print('Output name is: ', output_name)


Output name is:  output


## Question 2: Target size
(200,200)

## Question 3

Now we need to turn the image into numpy array and pre-process it.

After the pre-processing, what's the value in the first pixel, the R channel?

In [61]:
# download the images, resizing them with pillow

!pip install pillow

from io import BytesIO
from urllib import request
from PIL import Image
import numpy as np


def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img


In [62]:
# Note that the following pre-processing was done in the previous homework
'''
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])
'''

# Convert to array and preprocess

def preprocess_input(img):
    x = np.array(img, dtype='float32')
    
    # Normalize to [0, 1]
    x = x / 255.0
    
    # Normalize with ImageNet mean and std
    mean = np.array([0.485, 0.456, 0.406], dtype='float32')
    std = np.array([0.229, 0.224, 0.225], dtype='float32')
    x = (x - mean) / std
    
    # Convert from (H, W, C) to (C, H, W) - channels first format
    x = x.transpose(2, 0, 1)
    
    # Add batch dimension: (C, H, W) -> (1, C, H, W)
    X = np.array([x])
    
    return X


In [65]:


# Run the model with the image https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg

imgurl = 'https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg'
img = download_image(imgurl)
img = prepare_image(img, (200, 200))

X = preprocess_input(img)

# X shape should be (1, 3, 200, 200) - batch, channels, height, width
print(f"X shape: {X.shape}")

# First value of R channel (channel 0, first pixel)
print("First value of R: ", X[0, 0, 0, 0])

X shape: (1, 3, 200, 200)
First value of R:  -1.073294


## Question 4
Now let's apply this model to this image. What's the output of the model?

In [66]:
result = session.run([output_name], {input_name: X})
predictions = result[0][0].tolist()

print(predictions)


[0.09156588464975357]
